In [1]:
import json
from pathlib import Path
from collections import defaultdict

In [2]:
# Charger le fichier original
input_file = 'prix-carburants-quotidien.json'
output_file = 'prix-carburants-quotidien-preprocessed.json'

print(f"Lecture de {input_file}...")
with open(input_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

print(f"Nombre d'entrées chargées: {len(data) if isinstance(data, list) else 'N/A'}")

Lecture de prix-carburants-quotidien.json...
Nombre d'entrées chargées: 74592


In [3]:
# Exemple d'horaire avant transformation
if isinstance(data, list) and len(data) > 0:
    for item in data[:10]:  # Chercher un exemple avec horaires
        if 'horaires' in item and item['horaires']:
            print("Exemple d'horaire AVANT:")
            print(f"Type: {type(item['horaires'])}")
            print(f"Contenu: {item['horaires'][:200]}...")
            break

Exemple d'horaire AVANT:
Type: <class 'str'>
Contenu: {"@automate-24-24": "1", "jour": [{"@id": "1", "@nom": "Lundi", "@ferme": ""}, {"@id": "2", "@nom": "Mardi", "@ferme": ""}, {"@id": "3", "@nom": "Mercredi", "@ferme": ""}, {"@id": "4", "@nom": "Jeudi"...


In [4]:
def convert_time_format(time_str):
    """
    Convertit le format "06.30" ou "6.30" vers "06:30:00" (xsd:time compatible)
    """
    if not time_str:
        return None
    
    try:
        # Remplacer le point par deux-points
        time_str = time_str.replace('.', ':')
        
        # Séparer heures et minutes
        parts = time_str.split(':')
        if len(parts) == 2:
            hours = parts[0].zfill(2)  # Ajouter 0 devant si nécessaire
            minutes = parts[1].zfill(2)
            return f"{hours}:{minutes}:00"
        return None
    except Exception:
        return None


def generate_horaire_id(ouverture, fermeture, ferme):
    """
    Génère un ID unique pour une combinaison d'horaires.
    Permet de réutiliser les mêmes instances d'horaires.
    """
    if ferme == "1":
        return "Horaire_Ferme"
    
    if not ouverture or not fermeture:
        return "Horaire_NonDefini"
    
    # Simplifier les heures pour l'ID (enlever les secondes)
    ouv_simple = ouverture.rsplit(':', 1)[0].replace(':', 'h')
    ferm_simple = fermeture.rsplit(':', 1)[0].replace(':', 'h')
    
    return f"Horaire_{ouv_simple}_{ferm_simple}"


def parse_horaires(horaires_str):
    """
    Convertit une string JSON échappée en objet JSON.
    Si la conversion échoue, retourne None.
    """
    if not horaires_str or horaires_str == "":
        return None
    
    if isinstance(horaires_str, dict):
        return horaires_str
    
    try:
        return json.loads(horaires_str)
    except (json.JSONDecodeError, TypeError) as e:
        print(f"Erreur de parsing: {e}")
        return None

In [5]:
# Transformer toutes les entrées
processed = 0
errors = 0
horaire_ids_created = set()

if isinstance(data, list):
    for item in data:
        if 'horaires' in item:
            # 1. Parser la string JSON
            parsed_horaires = parse_horaires(item['horaires'])
            
            if parsed_horaires is not None and 'jour' in parsed_horaires:
                # 2. Traiter chaque jour
                for jour in parsed_horaires['jour']:
                    # Ajouter l'ID de la station
                    jour['station_id'] = item.get('id', '')
                    
                    # Convertir les heures si présentes
                    if 'horaire' in jour and isinstance(jour['horaire'], dict):
                        ouverture_orig = jour['horaire'].get('@ouverture', '')
                        fermeture_orig = jour['horaire'].get('@fermeture', '')
                        
                        ouverture_conv = convert_time_format(ouverture_orig)
                        fermeture_conv = convert_time_format(fermeture_orig)
                        
                        if ouverture_conv:
                            jour['horaire']['@ouverture'] = ouverture_conv
                        if fermeture_conv:
                            jour['horaire']['@fermeture'] = fermeture_conv
                        
                        # Générer un ID d'horaire
                        horaire_id = generate_horaire_id(
                            ouverture_conv, 
                            fermeture_conv, 
                            jour.get('@ferme', '')
                        )
                        jour['horaire_id'] = horaire_id
                        horaire_ids_created.add(horaire_id)
                    else:
                        # Jour fermé ou sans horaires
                        ferme = jour.get('@ferme', '')
                        if ferme == "1":
                            jour['horaire_id'] = "Horaire_Ferme"
                            horaire_ids_created.add("Horaire_Ferme")
                        else:
                            jour['horaire_id'] = "Horaire_NonDefini"
                            horaire_ids_created.add("Horaire_NonDefini")
                
                item['horaires'] = parsed_horaires
                processed += 1
            elif item['horaires'] != "" and item['horaires'] is not None:
                errors += 1
else:
    # Si c'est un dictionnaire avec des clés
    for key, item in data.items():
        if isinstance(item, dict) and 'horaires' in item:
            parsed_horaires = parse_horaires(item['horaires'])
            
            if parsed_horaires is not None and 'jour' in parsed_horaires:
                for jour in parsed_horaires['jour']:
                    jour['station_id'] = item.get('id', '')
                    
                    if 'horaire' in jour and isinstance(jour['horaire'], dict):
                        ouverture_orig = jour['horaire'].get('@ouverture', '')
                        fermeture_orig = jour['horaire'].get('@fermeture', '')
                        
                        ouverture_conv = convert_time_format(ouverture_orig)
                        fermeture_conv = convert_time_format(fermeture_orig)
                        
                        if ouverture_conv:
                            jour['horaire']['@ouverture'] = ouverture_conv
                        if fermeture_conv:
                            jour['horaire']['@fermeture'] = fermeture_conv
                        
                        horaire_id = generate_horaire_id(
                            ouverture_conv, 
                            fermeture_conv, 
                            jour.get('@ferme', '')
                        )
                        jour['horaire_id'] = horaire_id
                        horaire_ids_created.add(horaire_id)
                    else:
                        ferme = jour.get('@ferme', '')
                        if ferme == "1":
                            jour['horaire_id'] = "Horaire_Ferme"
                            horaire_ids_created.add("Horaire_Ferme")
                        else:
                            jour['horaire_id'] = "Horaire_NonDefini"
                            horaire_ids_created.add("Horaire_NonDefini")
                
                item['horaires'] = parsed_horaires
                processed += 1
            elif item['horaires'] != "" and item['horaires'] is not None:
                errors += 1

print(f"Horaires transformés: {processed}")
print(f"Erreurs rencontrées: {errors}")
print(f"IDs d'horaires uniques créés: {len(horaire_ids_created)}")

Horaires transformés: 61851
Erreurs rencontrées: 0
IDs d'horaires uniques créés: 402


In [6]:
print("Exemples d'IDs d'horaires générés:")
for horaire_id in sorted(list(horaire_ids_created))[:15]:
    print(f"  - {horaire_id}")

Exemples d'IDs d'horaires générés:
  - Horaire_00h00_00h00
  - Horaire_00h00_00h01
  - Horaire_00h00_00h59
  - Horaire_00h00_01h00
  - Horaire_00h00_06h30
  - Horaire_00h00_07h59
  - Horaire_00h00_14h00
  - Horaire_00h00_17h00
  - Horaire_00h00_19h30
  - Horaire_00h00_22h59
  - Horaire_00h00_23h00
  - Horaire_00h00_23h58
  - Horaire_00h00_23h59
  - Horaire_00h01_00h59
  - Horaire_00h01_22h58


In [7]:
# Exemple d'horaire après transformation
if isinstance(data, list) and len(data) > 0:
    for item in data[:50]:
        if 'horaires' in item and item['horaires'] and 'jour' in item['horaires']:
            # Trouver un exemple avec des horaires définis
            has_horaire = any('horaire' in j for j in item['horaires']['jour'])
            if has_horaire:
                print("Exemple d'horaire APRÈS transformation:")
                print(f"Station ID: {item.get('id', 'N/A')}")
                print(f"Type: {type(item['horaires'])}")
                print(json.dumps(item['horaires'], indent=2, ensure_ascii=False))
                break

Exemple d'horaire APRÈS transformation:
Station ID: 84000020
Type: <class 'dict'>
{
  "@automate-24-24": "1",
  "jour": [
    {
      "@id": "1",
      "@nom": "Lundi",
      "@ferme": "",
      "horaire": {
        "@ouverture": "06:00:00",
        "@fermeture": "20:00:00"
      },
      "station_id": "84000020",
      "horaire_id": "Horaire_06h00_20h00"
    },
    {
      "@id": "2",
      "@nom": "Mardi",
      "@ferme": "",
      "horaire": {
        "@ouverture": "06:00:00",
        "@fermeture": "20:00:00"
      },
      "station_id": "84000020",
      "horaire_id": "Horaire_06h00_20h00"
    },
    {
      "@id": "3",
      "@nom": "Mercredi",
      "@ferme": "",
      "horaire": {
        "@ouverture": "06:00:00",
        "@fermeture": "20:00:00"
      },
      "station_id": "84000020",
      "horaire_id": "Horaire_06h00_20h00"
    },
    {
      "@id": "4",
      "@nom": "Jeudi",
      "@ferme": "",
      "horaire": {
        "@ouverture": "06:00:00",
        "@fermeture": "20:

In [8]:
output_file

'prix-carburants-quotidien-preprocessed.json'

In [9]:
# Sauvegarder le fichier transformé
print(f"Sauvegarde dans {output_file}...")
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print("✓ Fichier sauvegardé avec succès!")

Sauvegarde dans prix-carburants-quotidien-preprocessed.json...
✓ Fichier sauvegardé avec succès!


In [10]:
# Vérification de la taille des fichiers
input_size = Path(input_file).stat().st_size / (1024 * 1024)  # MB
output_size = Path(output_file).stat().st_size / (1024 * 1024)  # MB

print(f"Taille fichier original: {input_size:.2f} MB")
print(f"Taille fichier nettoyé: {output_size:.2f} MB")
print(f"Différence: {output_size - input_size:.2f} MB ({((output_size/input_size - 1) * 100):.1f}%)")

Taille fichier original: 110.33 MB
Taille fichier nettoyé: 187.13 MB
Différence: 76.79 MB (69.6%)
